von Fabian Wolter und Selin Kabak
# Kalibrierung von Digitalkameras
### Versuch 2, Praktikum Technische Grundlagen der angewandten Informatik

TODO (1) Aufnahme und Analyse eines Grauwertkeiles
: 
- Speichern Sie das Bild für die Weiterverarbeitung und für das Notebook (Befehl: cv2.imwrite()) im verlustfreien Format png ab.
- Schreiben Sie eine Routine, die das Bild einliest (Befehl: pyopencv.imread()) und die einzelnen Grauwertstufen als Unterbilder aus dem Originalbild ausliest.
- Bestimmen Sie für jede Stufe den Mittelwert und die Standardabweichung der Grauwerte und tragen Sie diese in eine Tabelle ein.

In [ ]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)

# capture frames from the camera
while True:
    ret, frame = cap.read() 
    
    # Convert the frame to grayscale
    grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('Grayscale', grayscale)

    # Break the loop if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        # Save 10 images after exiting the loop
        for i in range(10):
            ret, frame = cap.read()  # Read another frame
            grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            cv.imwrite(f'weiss/weissbild{i}.png', grayscale) 
            cv.waitKey(1)  # Brief delay
        break 

# Release the camera and close any OpenCV windows
cap.release()
cv.destroyAllWindows()

# Number of grayscale levels
num_levels = 10
level_stats = []

# Read each saved image to calculate the mean and standard deviation for each sub-image
for i in range(10):
    image = cv.imread(f'weiss/weissbild{i}.png', cv.IMREAD_GRAYSCALE) 
    # Split the image into num_levels sub-images based on height
    level_height = image.shape[0] // num_levels  # Calculate height for each level
    for j in range(num_levels):
        # Extract sub-image for each gray level
        sub_image = image[j * level_height: (j + 1) * level_height, :]
        
        # Calculate mean and standard deviation
        mean = np.mean(sub_image)
        stddev = np.std(sub_image)  
        level_stats.append((mean, stddev))

for i, (mean, stddev) in enumerate(level_stats):
    image_num = i // num_levels + 1
    level_num = i % num_levels + 1
    print(f'Image {image_num}, Level {level_num}: Mean = {mean:.2f}, StdDev = {stddev:.2f}')

TODO (2) Aufnahme eines Dunkelbildes
: 
- Schreiben Sie eine Routine, die die 10 Bilder einliest, in double umwandelt, die Farbbilder in Grauwertbilder umrechnet und ihren pixelweisen Mittelwert berechnet. Stellen Sie das Dunkelbild kontrastmaximiert in Ihrem Notebook dar.
- Schreiben Sie eine Routine, die das Dunkelbild einliest, von einem Eingabebild subtrahiert und das korrigierte Bild abspeichert.

In [ ]:
import cv2 as cv
import numpy as np

grayscale_images = []

for i in range(10):
    img = cv.imread(f'schwarz/schwarzbild{i}.png')
    grayscale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY).astype(float)  # Convert to grayscale
    grayscale_images.append(grayscale_img)  # Store in the list

mean_image = np.mean(grayscale_images, axis=0)  

# contrast maximized
normalized_mean_image = mean_image / np.max(mean_image)
cv.imshow('Mean Image (Normalized)', normalized_mean_image)
cv.waitKey(0)  # Wait for a key press to close the window
cv.destroyAllWindows() 

cv.imwrite('dark_image.png', mean_image.astype(np.uint8))

# Subtract the Dark Image from an Input Image
def subtract_dark_image(input_image_path, dark_image_path, output_image_path):
    input_image = cv.imread(input_image_path).astype(float)
    dark_image = cv.imread(dark_image_path).astype(float)

    # Subtract the dark image from the input image
    corrected_image = cv.subtract(input_image, dark_image)

    # Convert corrected image to uint8
    corrected_image_uint8 = np.clip(corrected_image, 0, 255).astype(np.uint8)  
    cv.imwrite(output_image_path, corrected_image_uint8)

subtract_dark_image('input_image.png', 'dark_image.png', 'corrected_image.png')

TODO (3) Aufnahme eines Weissbildes
: 
- Schreiben Sie eine Routine, die die 10 Bilder einliest und ihren Mittelwert berechnet. Subtrahieren Sie von dem Mittelwertbild das Dunkelbild und speichern Sie das resultierende Weissbild ab. Stellen Sie das Weissbild kontrastmaximiert dar.
- Erweitern Sie die Routine zur Subtraktion des Dunkelbildes aus Aufgabe 2 so, dass das Weissbild eingelesen, in double umgewandelt wird und so normiert wird, dass sein Mittelwert 1 ist. Das durch Abzug des Dunkelbildes korrigierte Eingangsbild wird anschliessend durch das normierte Weissbild dividiert. 

In [ ]:
import cv2
import numpy as np

def calculate_white_image(white_images_path, dark_image_path, output_path):
    imgarr = []
    for i in range(10):
        img = cv2.imread(f"{white_images_path}/weissbild{i}.png", cv2.IMREAD_GRAYSCALE).astype(np.double)
        imgarr.append(img)

    mean_image = np.mean(imgarr, axis=0)

    dark_image = cv2.imread(dark_image_path, cv2.IMREAD_GRAYSCALE).astype(np.double)

    # Calculate white image
    white_image = mean_image - dark_image
    cv2.imwrite(f"{output_path}/white_image.png", white_image.astype(np.uint8))

    # contrast maximized
    normalized_white_image = white_image / np.max(white_image)
    cv2.imshow('White Image', normalized_white_image) 
    cv2.waitKey(0)  # Keep the window open until a key is pressed
    cv2.destroyAllWindows()
    return white_image 

# correct an input image by subtracting the dark image and dividing by the white image
def correct_image(input_image_path, dark_image_path, white_image, output_path):
    input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE).astype(np.double)
    dark_image = cv2.imread(dark_image_path, cv2.IMREAD_GRAYSCALE).astype(np.double)

    corrected_image = (input_image - dark_image) / white_image 

    # Clip to [0, 255] and convert to uint8
    corrected_image_uint8 = np.clip(corrected_image, 0, 255).astype(np.uint8)
    cv2.imwrite(f"{output_path}/corrected_image.png", corrected_image_uint8)

# Paths
white_images_path = "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/weiss"
dark_image_path = "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed/darkmean.png"
output_path = "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed"

# Calculate the white image
white_image = calculate_white_image(white_images_path, dark_image_path, output_path)

# example
correct_image("/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/weiss/weissbild0.png", dark_image_path, white_image, output_path)

TODO (4) Pixelfehler
: 
- Korrigieren Sie mithilfe Ihres Programms aus Aufgabe 3 das Bild des Grauwertkeils aus Aufgabe1.
- Werten Sie nun erneut das korrigierte Bild aus Aufgabe4 mit diesem Programm aus und erstellen Sie die zugehoerige Tabelle. Ergibt sich eine Verbesserung?

In [ ]:
import cv2
import numpy as np

# mark hot pixels or dead pixels
def mark_pixels(image_path, threshold, color):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE).astype(np.double)
    
    # Convert to BGR for marking in color
    marked_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2BGR)

    if color == 'red':  # For hot pixels
        marked_image[image > threshold] = [0, 0, 255]  
    elif color == 'blue':  # For dead pixels
        marked_image[image < threshold] = [255, 0, 0]

    # Save the marked image with a new name
    output_path = image_path.replace('.png', '_marked.png')
    cv2.imwrite(output_path, marked_image)

# Mark hot pixels in the dark image and dead pixels in the white image
mark_pixels("/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed/darkmean.png", 50, 'red')
mark_pixels("/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed/white_image.png", 50, 'blue')

# Correct the grayscale wedge image using the calculated dark and white images from the previous task
correct_image(
    "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/grayscale_wedge.png",
    "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed/darkmean.png",
    white_image,
    "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed"
)

# Read the corrected image to evaluate the mean and standard deviation
corrected_image_path = "/home/selin/HTWG-AIN/AIN3/sss/versuch2/Kamera2/changed/corrected_image.png"
corrected_image = cv2.imread(corrected_image_path, cv2.IMREAD_GRAYSCALE)

if corrected_image is None:
    print(f"Error: Could not read the corrected image at {corrected_image_path}")
else:
    mean = np.mean(corrected_image)
    stddev = np.std(corrected_image) 
    
    print(f'Corrected Image: Mean = {mean:.2f}, StdDev = {stddev:.2f}')